In [1]:
%matplotlib inline
# Importing standard Qiskit libraries
from qiskit import QuantumCircuit, execute, Aer, IBMQ
from qiskit.compiler import transpile, assemble
from qiskit.tools.jupyter import *
from qiskit.visualization import *
from ibm_quantum_widgets import *
from math import gcd
from numpy.random import randint
import pandas as pd
from fractions import Fraction
import matplotlib.pyplot as plt
import numpy as np

# Loading your IBM Q account(s)
provider = IBMQ.load_account()

In [2]:
def c_amod14(a, power, prepower):
    
    """Controlled multiplication by a mod 14"""
    if a not in [3,5,9,11,13]:
        raise ValueError("'a' must be 3,5,9,11,13")
    
    U = QuantumCircuit(4)
    if prepower<1:
        prepower=1
    for i in range(power-prepower, power+1):
        if a == 3:
            x = i%6
            if x == 0:
                U.x(2)
            elif x == 1:
                U.x(0)
                U.x(2)
            elif x == 2:
                U.x(1)
            elif x == 3:
                U.x(1)
                U.x(2)
            elif x == 4:
                U.x(0)
                U.x(1)
                U.x(2)
            elif x == 5:
                U.x(1)
        if a == 5:
            x = i%6
            if x == 0:
                U.x(1)
            elif x == 1:
                U.x(0)
                U.x(1)
                U.x(2)
            elif x == 2:
                U.x(1)
                U.x(2)
            elif x == 3:
                U.x(0)
            elif x == 4:
                U.x(1)
                U.x(2)
            elif x == 5:
                U.x(2)
        if a == 9:
            x = i%3
            if x==0:
                U.x(1)
            elif x==1:
                U.x(0)
                U.x(1)
                U.x(2)
            elif x==2:
                U.x(0)
                U.x(2)
            
        if a == 13:
            x = i%2
            if x==0:
                U.x(0)
                U.x(1)
            else:
                U.x(0)
                U.x(1)
        if a == 11:
            x = i%3
            if x==0:
                U.x(0)
                U.x(2)
            if x==1:
                U.x(0)
                U.x(1)
                U.x(2)
            if x==2:
                U.x(1)
    U = U.to_gate()
    U.name = "%i^%i mod 14" % (a, power)
    c_U = U.control()
    return c_U

In [3]:
def qpe_amod14(a):
    #2n^2<=q<=3n^2 --> q = 2^n_count = 256
    #overall 512 different states
    n_count = 8 
    qc = QuantumCircuit(4+n_count, n_count)
    for q in range(n_count):
        qc.h(q)     # Initialize counting qubits in state |+>
    qc.x(3+n_count) # And auxiliary register in state |1>
    for q in range(n_count): # Do controlled-U operations
        qc.append(c_amod14(a, 2**q, 2**(q-1)), [q] + [i+n_count for i in range(4)])
    qc.append(qft_dagger(n_count), range(n_count)) # Do inverse-QFT
    qc.measure(range(n_count), range(n_count))
    qc.draw(fold=-1)
    # Simulate Results
    aer_sim = Aer.get_backend('aer_simulator')
    # Setting memory=True below allows us to see a list of each sequential reading
    t_qc = transpile(qc, aer_sim)
    qobj = assemble(t_qc, shots=1)
    result = aer_sim.run(qobj, memory=True).result()
    readings = result.get_memory()
    print("Register Reading: " + readings[0])
    phase = int(readings[0],2)/(2**n_count)
    print("Corresponding Phase: %f" % phase)
    return phase

In [4]:
def qft_dagger(n):
    
    """n-qubit QFTdagger the first n qubits in circ"""
    qc = QuantumCircuit(n)
    
    # Don't forget the Swaps!
    for qubit in range(n//2):
        qc.swap(qubit, n-qubit-1)
    
    for j in range(n):
        for m in range(j):
            qc.cp(-np.pi/float(2**(j-m)), m, j)
        qc.h(j)
    
    qc.name = "QFT†"
    return qc

In [5]:
#a can take values [3,5,9,11,13]
#Change only to these numbers
a = 3
factor_found = False
attempt = 0
N = 14
while not factor_found:
    attempt += 1
    print("\nAttempt %i:" % attempt)
    #Phase = s(1/r)
    phase = qpe_amod14(a)
    #Get fraction from float
    frac = Fraction(phase).limit_denominator(N)
    #r = denominator of frac
    r = frac.denominator
    print("Result: r = %i" % r)
    if phase != 0:
        # Guesses for factors are gcd(x^{r/2} ±1 , 14)
        guesses = [gcd(a**(r//2)-1, N), gcd(a**(r//2)+1, N)]
        print("Guessed Factors: %i and %i" % (guesses[0], guesses[1]))
        for guess in guesses:
            #If guess is facor of N
            if guess not in [1,N] and (N % guess) == 0:
                print("Non-trivial factor found: %i" % guess)
                factor_found = True


Attempt 1:
Register Reading: 10100101
Corresponding Phase: 0.644531
Result: r = 14
Guessed Factors: 2 and 2
Non-trivial factor found: 2
Non-trivial factor found: 2


/opt/conda/lib/python3.8/site-packages/qiskit/utils/deprecation.py:62: DeprecationWarning: Using a qobj for run() is deprecated as of qiskit-aer 0.9.0 and will be removed no sooner than 3 months from that release date. Transpiled circuits should now be passed directly using `backend.run(circuits, **run_options).
  return func(*args, **kwargs)
